In [ ]:
import random
import os
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from tqdm import tqdm
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from PIL import Image
import json

### Look at annotation file

In [ ]:
import json

# Path to COCO annotation file
with open("/home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N5-1.json", "r") as f:
    coco_data = json.load(f)

# Inspect keys
print(coco_data.keys())

In [ ]:
coco_data["images"]

In [ ]:
target_category_id = 18

# Filter annotations
filtered_anns = [
    ann for ann in coco_data["annotations"]
    if ann["category_id"] == target_category_id
]

print(f"Found {len(filtered_anns)} annotations with category_id={target_category_id}\n")
for ann in filtered_anns:  # show first 5
    print(ann)

### New crop generation
Make sure to get crops from all boxes in the mask, or generate crops directly from .json

In [ ]:
# train
dataset = 'nwpu'
for N in [1, 3, 5, 10, 30]:
    for M in [1, 2, 3]:
        # === Paths ===
        images_dir = f"/home/gridsan/manderson/ovdsat/data/{dataset}/positive_image_set"
        annotations_path = f"/home/gridsan/manderson/ovdsat/data/{dataset}/train_coco_subset_N{N}-{M}.json"
        output_dir = f"/home/gridsan/manderson/ovdsat/data/cropped_data/{dataset}/train/{dataset}_N{N}-{M}"

        # === Load annotations ===
        with open(annotations_path, "r") as f:
            coco = json.load(f)

        # Build category lookup
        categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

        # Make output dirs per category
        for cat_name in categories.values():
            os.makedirs(os.path.join(output_dir, cat_name), exist_ok=True)

        # Build lookup for image_id -> filename
        image_lookup = {img["id"]: img["file_name"] for img in coco["images"]}

        # === Loop through annotations ===
        for ann in tqdm(coco["annotations"]):
            image_id = ann["image_id"]
            cat_id = ann["category_id"]
            bbox = ann["bbox"]  # format: [x, y, width, height]

            # Load image
            img_path = os.path.join(images_dir, image_lookup[image_id])
            if not os.path.exists(img_path):
                continue  # skip if missing image
            img = Image.open(img_path).convert("RGB")

            # Crop bounding box
            x, y, w, h = map(int, bbox)
            crop = img.crop((x, y, x + w, y + h))

            # Save into category folder
            cat_name = categories[cat_id]
            save_dir = os.path.join(output_dir, cat_name)
            save_path = os.path.join(save_dir, f"{image_id}_{ann['id']}.jpg")
            crop.save(save_path)

        print(f"Crops generated and saved per category for N={N}, M={M}!")

In [ ]:
# val

for M in [1, 2, 3]:
    # === Paths ===
    images_dir = f"/home/gridsan/manderson/ovdsat/data/{dataset}/positive_image_set"               # folder where your dataset images are stored
    annotations_path = f"/home/gridsan/manderson/ovdsat/data/{dataset}/val_coco-{M}.json"
    output_dir = f"/home/gridsan/manderson/ovdsat/data/cropped_data/{dataset}/val/{dataset}_val-{M}"

    # === Load annotations ===
    with open(annotations_path, "r") as f:
        coco = json.load(f)

    # Build category lookup
    categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

    # Make output dirs per category
    for cat_name in categories.values():
        os.makedirs(os.path.join(output_dir, cat_name), exist_ok=True)

    # Build lookup for image_id -> filename
    image_lookup = {img["id"]: img["file_name"] for img in coco["images"]}

    # === Loop through annotations ===
    for ann in tqdm(coco["annotations"]):
        image_id = ann["image_id"]
        cat_id = ann["category_id"]
        bbox = ann["bbox"]  # format: [x, y, width, height]

        # Load image
        img_path = os.path.join(images_dir, image_lookup[image_id])
        if not os.path.exists(img_path):
            continue  # skip if missing image
        img = Image.open(img_path).convert("RGB")

        # Crop bounding box
        x, y, w, h = map(int, bbox)
        crop = img.crop((x, y, x + w, y + h))

        # Save into category folder
        cat_name = categories[cat_id]
        save_dir = os.path.join(output_dir, cat_name)
        save_path = os.path.join(save_dir, f"{image_id}_{ann['id']}.jpg")
        crop.save(save_path)

    print(f"Crops generated and saved per category for M={M}!")

In [ ]:
# Check number of crops in each
def count_direct_files(directory):
    directory = Path(directory)
    return {
        d.name: sum(1 for f in d.iterdir() if f.is_file())
        for d in directory.iterdir() if d.is_dir() and 'checkpoints' not in d.name
    }

# Example usage:
directory_path = f"/home/gridsan/manderson/ovdsat/data/cropped_data/{dataset}/train/{dataset}_N5-3"
counts = count_direct_files(directory_path)

for subdir, count in counts.items():
    print(f"{count} files: {subdir}")

In [ ]:
def show_random_images_per_subdir(root_dir, n=3):
    """
    Iterate through subdirectories of root_dir, and for each subdir containing
    .jpg files, print the subdir name and display n random images.
    """
    for subdir, _, files in os.walk(root_dir):
        jpg_files = [f for f in files if f.lower().endswith(".jpg")]
        if not jpg_files or '.ipynb_checkpoints' in subdir:
            continue  # skip if no jpg files

        print(f"\nSubdir: {os.path.basename(subdir)} ({len(jpg_files)} total crops)")

        # Pick random files
        selected_files = random.sample(jpg_files, min(n, len(jpg_files)))

        # Plot them
        plt.figure(figsize=(15, 5))
        for i, file in enumerate(selected_files, 1):
            img_path = os.path.join(subdir, file)
            img = Image.open(img_path)

            plt.subplot(1, len(selected_files), i)
            plt.imshow(img)
            plt.axis("off")
            plt.title(file, fontsize=8)

        plt.tight_layout()
        plt.show()

In [ ]:
show_random_images_per_subdir("/home/gridsan/manderson/ovdsat/data/cropped_data/nwpu/train/nwpu_N5-1", n=5)

In [ ]:
show_random_images_per_subdir("/home/gridsan/manderson/ovdsat/data/cropped_data/nwpu/val/nwpu_val-1", n=5)

In [ ]:
M = 1
for cat in os.listdir(f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N1-{M}'):
    print(cat)
    for N in [1, 3, 5, 10, 20, 30]:
        print(N)
        path = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N{N}-{M}/{cat}'
        files = os.listdir(path)
        files.sort()
        print(files)
        for img_name in files:
            if 'ipynb' not in img_name:
                img = Image.open(f'{path}/{img_name}')
                plt.imshow(img)
                plt.show()
    print()

### Background crops
Change this so it generates directly from json annotation files

In [ ]:
def generate_background_boxes_from_mask(mask_array, num_b, min_size=30, max_size=200, max_iter=100):
    """
    Generate boxes that do NOT intersect with non-zero mask areas.
    """
    h, w = mask_array.shape
    boxes = []

    for _ in range(num_b):
        valid_box = False
        count = 0
        while not valid_box and count < max_iter:
            count += 1
            width = random.randint(min_size, min(max_size, w))
            height = random.randint(min_size, min(max_size, h))
            x = random.randint(0, w - width)
            y = random.randint(0, h - height)
            x2 = x + width
            y2 = y + height

            # Check if the box intersects with foreground mask
            if np.any(mask_array[y:y2, x:x2] > 0):
                continue  # intersecting: skip
            boxes.append((x, y, x2, y2))
            valid_box = True

    return boxes

def generate_background_crops_from_masks(input_dir, output_root, num_b=10, min_size=30, max_size=200):
    input_path = Path(input_dir)
    background_dir = Path(output_root) / "background"
    background_dir.mkdir(parents=True, exist_ok=True)

    for root, _, files in os.walk(input_path):
        for file in tqdm(files):
            if file.endswith(".jpg") and not file.endswith(".mask.jpg"):
                image_file = Path(root) / file
                mask_file = image_file.with_name(f"{image_file.stem}.mask.jpg")

                if not mask_file.exists():
                    print(f"[!] No mask found for {image_file.name}, skipping.")
                    continue

                try:
                    image = Image.open(image_file).convert("RGB")
                    mask = Image.open(mask_file).convert("L")
                except Exception as e:
                    print(f"[✗] Error loading {image_file.name}: {e}")
                    continue

                mask_array = np.array(mask)
                boxes = generate_background_boxes_from_mask(mask_array, num_b, min_size, max_size)

                for i, (x1, y1, x2, y2) in enumerate(boxes):
                    crop = image.crop((x1, y1, x2, y2))
                    crop_filename = f"{image_file.stem}_background{i}.jpg"
                    crop.save(background_dir / crop_filename)
                    #print(f"[✓] Saved: {background_dir / crop_filename}")

In [ ]:
# For train

for N in [1, 3, 5, 10, 30]:
    for M in [1]:
        print(f'================ Processing train N={N}, M={M} ================')
        input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N{N}-{M}'
        generate_background_crops_from_masks(input_root, output_root)
        print()

In [ ]:
# For val

for M in [1]:
    print(f'================ Processing val M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-{M}'
    generate_background_crops_from_masks(input_root, output_root)
    print()

### Sanity check

In [ ]:
# train

for N in [100]:
    for M in [1, 2, 3]:
        # === Paths ===
        images_dir = "/home/gridsan/manderson/ovdsat/data/dior/JPEGImages"
        annotations_path = f"/home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N{N}-{M}.json"
        output_dir = f"/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N{N}-{M}"

        # === Load annotations ===
        with open(annotations_path, "r") as f:
            coco = json.load(f)

        # Build category lookup
        categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

        # Make output dirs per category
        for cat_name in categories.values():
            os.makedirs(os.path.join(output_dir, cat_name), exist_ok=True)

        # Build lookup for image_id -> filename
        image_lookup = {img["id"]: img["file_name"] for img in coco["images"]}

        # === Loop through annotations ===
        for ann in tqdm(coco["annotations"]):
            image_id = ann["image_id"]
            cat_id = ann["category_id"]
            bbox = ann["bbox"]  # format: [x, y, width, height]

            # Load image
            img_path = os.path.join(images_dir, image_lookup[image_id])
            if not os.path.exists(img_path):
                continue  # skip if missing image
            img = Image.open(img_path).convert("RGB")

            # Crop bounding box
            x, y, w, h = map(int, bbox)
            crop = img.crop((x, y, x + w, y + h))

            # Save into category folder
            cat_name = categories[cat_id]
            save_dir = os.path.join(output_dir, cat_name)
            save_path = os.path.join(save_dir, f"{image_id}_{ann['id']}.jpg")
            crop.save(save_path)

        print(f"Crops generated and saved per category for N={N}, M={M}!")

In [ ]:
# val

N=100
for M in [1, 2, 3]:
    # === Paths ===
    images_dir = "/home/gridsan/manderson/ovdsat/data/dior/JPEGImages"               # folder where your dataset images are stored
    annotations_path = f"/home/gridsan/manderson/ovdsat/data/dior/val_coco_N{N}-{M}.json"
    output_dir = f"/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val_N{N}-{M}"

    # === Load annotations ===
    with open(annotations_path, "r") as f:
        coco = json.load(f)

    # Build category lookup
    categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

    # Make output dirs per category
    for cat_name in categories.values():
        os.makedirs(os.path.join(output_dir, cat_name), exist_ok=True)

    # Build lookup for image_id -> filename
    image_lookup = {img["id"]: img["file_name"] for img in coco["images"]}

    # === Loop through annotations ===
    for ann in tqdm(coco["annotations"]):
        image_id = ann["image_id"]
        cat_id = ann["category_id"]
        bbox = ann["bbox"]  # format: [x, y, width, height]

        # Load image
        img_path = os.path.join(images_dir, image_lookup[image_id])
        if not os.path.exists(img_path):
            continue  # skip if missing image
        img = Image.open(img_path).convert("RGB")

        # Crop bounding box
        x, y, w, h = map(int, bbox)
        crop = img.crop((x, y, x + w, y + h))

        # Save into category folder
        cat_name = categories[cat_id]
        save_dir = os.path.join(output_dir, cat_name)
        save_path = os.path.join(save_dir, f"{image_id}_{ann['id']}.jpg")
        crop.save(save_path)

    print(f"Crops generated and saved per category for M={M}!")